In [ ]:
import sys
sys.path.insert(0, "..")

In [ ]:
import algo
import graph
import typing
from algo.min_cut import FlowGraph, FlowGraphEdge, FlowGraphNode, MAX_EDGE_CAPACITY

In [ ]:
g = graph.ExecutionGraph.load_all(open('dump.yaml'))[0]

In [ ]:
nodes: typing.Dict[str, FlowGraphNode] = {
    v.uuid: FlowGraphNode([]) for v in g.get_vertices()
}
edges: typing.List[FlowGraphEdge] = []
index = 0
for u, v, d in g.get_edges():
    bd = d["unit_size"] * d["per_second"]
    edges.append(FlowGraphEdge(u, v, bd, 0))
    nodes[u].out_edges.append(index)
    index += 1
    edges.append(FlowGraphEdge(v, u, 0, 0))
    nodes[v].out_edges.append(index)
    index += 1

fake_source = "fake_source"
fake_sink = "fake_sink"
nodes[fake_source] = FlowGraphNode([])
nodes[fake_sink] = FlowGraphNode([])
for s in g.get_in_vertices():
    edges.append(FlowGraphEdge(fake_source, s.uuid, MAX_EDGE_CAPACITY, 0))
    nodes[fake_source].out_edges.append(index)
    index += 1
    edges.append(FlowGraphEdge(s.uuid, fake_source, 0, 0))
    nodes[s.uuid].out_edges.append(index)
    index += 1
for s in g.get_out_vertices():
    edges.append(FlowGraphEdge(s.uuid, fake_sink, MAX_EDGE_CAPACITY, 0))
    nodes[s.uuid].out_edges.append(index)
    index += 1
    edges.append(FlowGraphEdge(fake_sink, s.uuid, 0, 0))
    nodes[fake_sink].out_edges.append(index)
    index += 1

flow_graph = FlowGraph(nodes, edges)

In [ ]:
len(g.g.edges()), len(g.get_sinks()), len(edges)

In [ ]:
for u, v in g.g.edges():
    print('{} ---> {}'.format(u, v))

In [ ]:
e_idxes = flow_graph.shortest_path("fake_source", "fake_sink")
print(e_idxes)
for idx in e_idxes:
    print(edges[idx].from_node, edges[idx].to_node, edges[idx].cap, edges[idx].flow)
min_incr = min([edges[idx].cap - edges[idx].flow for idx in e_idxes])
print(min_incr)
for idx in e_idxes:
    e = edges[idx]
    oe = edges[idx ^ 1]
    e.flow += min_incr
    oe.flow -= min_incr
    if e.cap == e.flow:
        e.disabled = True

In [ ]:
for e in edges:
    if e.disabled:
        print(e.from_node, e.to_node, e.cap, e.flow)

In [ ]:
flow_graph.reachable("fake_source")